## Mapping tutorial

In this tutorial we will explore the types of data that the MinION produces, and try to look at the error mode by visual inspection of alignments.

We will use data from the Ebola surveillance study in Guinea. Reads from this study are already on the server, in the tar archive ``076383.tgz``

However, first we need a reference sequence to map against. 

In [ ]:
wget https://raw.githubusercontent.com/nickloman/ebov/master/refs/EM_079517.fasta

Now let's extract FASTA files from the archive of reads. We can do this directly from the tar archive, but it's faster to extract the archive first. A tar archive is denoted by the file suffix 'tar' or 'tgz' for a zipped tar file. Poretools can extract a subset of reads using the '--type' parameter. Commonly used values include:

   * ``--type 2D``  (two-direction reads)
   * ``--type fwd`` (template reads)
   * ``--type rev`` (complement reads)
   
More detailed usage can be found at the documentation site:

<http://poretools.readthedocs.org/en/latest/content/examples.html#poretools-fastq>

In [ ]:
tar xvfz /data2/minion/R7/Ebola/076383.tgz

This makes a directory called ``076383_180Genomes_11rx``

Let's extract the reads:

In [ ]:
poretools fasta --type 2D 076383_180Genomes_11rx/pass > Ebola2D.fasta

Check how many reads you have obtained:

In [ ]:
grep ">" Ebola2D.fasta | wc -l

Now let's align the reads to a reference sequence. You need to index the reference first:

In [ ]:
bwa index EM_079517.fasta

Align the reads to the reference. Here we use BWA as the aligner and we specify that we have Oxford Nanopore reads with the ont2d option:

In [ ]:
bwa mem -x ont2d EM_079517.fasta Ebola2D.fasta | samtools view -bS - | samtools sort - -o Ebola2D.sorted.bam

This command makes a file called ``Ebola2D.sorted.bam``

Now index that bamfile:

In [ ]:
samtools index Ebola2D.sorted.bam

We can get some basic statistics about how well it has aligned using ``samtools stats``

In [ ]:
samtools stats Ebola2D.sorted.bam > Ebola2D.stats.txt

In [ ]:
head -40 Ebola2D.stats.txt

   - How many reads were mapped?
   - What is the average length of the reads?

``samtools stats`` can give us some coverage plots:

In [ ]:
grep "^COV" Ebola2D.stats.txt > Ebola2D.coverage.txt

In [ ]:
head -10 Ebola2D.coverage.txt

You can plot this in RStudio with code like this:

In [ ]:
library(ggplot2)
cov=read.table("Ebola2D.coverage.txt", sep="\t")
cov[1,]
ggplot(cov, aes(x=V3, y=V4)) + geom_bar(stat="identity") + xlab("Coverage") + ylab("Count")

What do you think about the coverage plot? What clues does it give you about how the sample was prepared?

### Consolidating your knowledge

Now, repeat this process from the beginning, but do it for a different dataset, choose from:

   - All 1D pass reads (hint: --type fwd,rev)
   - All pass forward reads (hint: --type fwd)
   - All pass reverse reads (hint: --type rev)
   - All 2D fail reads (hint: --type 2D, use the fail directory)

1D reads only! Ensure you use a different file name, e.g. ``Ebola1D.fasta``

   - How does the number of reads change?
   - How does the mapping frequency change?

## Inspecting alignments

Now, let's download the BAM file and inspect the alignment. My favoured tool for this is Tablet. It requires Java.

<https://ics.hutton.ac.uk/tablet/>

You need to load the following two files into Tablet:

alignment file: Ebola2D.sorted.bam
reference file: EM_079517.fasta

Inspect the alignment.

   - Did the alignment confirm your earlier suspicions about how the sample was prepared? 
   - What are the pros and cons of this approach?
   - Which regions might you be suspicious of?

Have a look at the error profile. Are some parts of the genome better than others? Can you correlate this with the sequence?

## Variant calling

The Ebola virus mutation rate is in the order of 1.2 x 10^-3 mutations/site/year. The genome size is 19000 bases long. This sample was collected about a year after the reference genome. Approximately how many SNPs do you expect to see?

Call SNPs - by eye!

   - Make a list of SNPs - which ones are hard to assess?

### Variant calling with nanopolish

Calling variants with nanopolish relies on squiggle data to generate the best consensus and gives a nicer result.

To call variants, there are three steps:

   - align the reads with BWA (or another aligner, such as marginAlign, or LAST)
   - align the events with ``nanopolish eventalign``
   - call a VCF with ``nanopolish variants``

We've already aligned the reads (output file from BWA was ``Ebola2D.sorted.bam``)

In [ ]:
nanopolish-r7 eventalign --reads Ebola2D.fasta -b Ebola2D.sorted.bam -g EM_079517.fasta --sam | samtools view -bS - | samtools sort - -o Ebola2D.eventalign.bam

We need to index the new BAM file that ``nanopolish eventalign`` produced:


In [ ]:
samtools index Ebola2D.eventalign.bam

And now we need to get the variants in VCF format:

In [ ]:
nanopolish-r7 variants --progress -t 1 --reads Ebola2D.fasta -o Ebola2D.vcf -b Ebola2D.sorted.bam -e Ebola2D.eventalign.bam -g EM_079517.fasta -vv -w "EM_079517:0-20000" --snp

It is actually possible to use different models with nanopolish variants specifying the model filenames ``--models-fofn offset_models.fofn``. In this case we swap the original 5-mer model for a 6-mer model.

Copy the model files into your current directory from: /data2/models/ into your current directory.

In [ ]:
cp /data2/models/* .

Compare this list with the list of variants that you already eyeballed. How do they compare?

Did nanopolish spot things that you didn't?

Did nanopolish get anything wrong? Could you figure out a way of filtering the VCF to remove these errors? 

## SNP calling with 6-mer model

In [ ]:
nanopolish variants --progress -t 1 --reads Ebola2D.fasta -o Ebola2D.6mer.vcf -b Ebola2D.sorted.bam -e Ebola2D.eventalign.bam -g EM_079517.fasta -vv -w "EM_079517:0-20000" --snp

   - How does the new VCF ``Ebola2D.6mer.vcf`` look compared with the old one?